In [2]:
import pandas as pd

df = pd.read_csv('../data/external/hw1-data.csv')


In [3]:
df

,User,"Gender (1 =F, 0=M)",260: Star Wars: Episode IV - A New Hope (1977),1210: Star Wars: Episode VI - Return of the Jedi (1983),356: Forrest Gump (1994),"318: Shawshank Redemption, The (1994)","593: Silence of the Lambs, The (1991)",3578: Gladiator (2000),1: Toy Story (1995),2028: Saving Private Ryan (1998),...,2396: Shakespeare in Love (1998),2916: Total Recall (1990),780: Independence Day (ID4) (1996),541: Blade Runner (1982),1265: Groundhog Day (1993),"2571: Matrix, The (1999)",527: Schindler's List (1993),"2762: Sixth Sense, The (1999)",1198: Raiders of the Lost Ark (1981),34: Babe (1995)
0,755,0,1.0,5.0,2.0,NaN,4.0,4.0,2.0,2.0,...,2.0,NaN,5.0,2.0,NaN,4.0,2.0,5.0,NaN,NaN
1,5277,0,5.0,3.0,NaN,2.0,4.0,2.0,1.0,NaN,...,3.0,2.0,2.0,NaN,2.0,NaN,5.0,1.0,3.0,NaN
2,1577,1,NaN,NaN,NaN,5.0,2.0,NaN,4.0,NaN,...,NaN,1.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0
3,4388,0,NaN,3.0,NaN,NaN,NaN,1.0,2.0,3.0,...,NaN,4.0,1.0,3.0,5.0,NaN,5.0,1.0,1.0,2.0
4,1202,1,4.0,3.0,4.0,1.0,4.0,1.0,NaN,4.0,...,5.0,1.0,NaN,4.0,NaN,3.0,5.0,5.0,NaN,NaN
5,3823,1,2.0,4.0,4.0,4.0,NaN,NaN,3.0,1.0,...,5.0,2.0,4.0,NaN,1.0,NaN,NaN,3.0,NaN,2.0
6,5448,0,NaN,NaN,3.0,1.0,1.0,4.0,NaN,5.0,...,1.0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,5.0,2.0
7,5347,0,4.0,NaN,NaN,NaN,3.0,2.0,2.0,NaN,...,NaN,2.0,3.0,2.0,4.0,NaN,1.0,3.0,5.0,NaN
8,4117,1,5.0,1.0,NaN,4.0,2.0,4.0,4.0,4.0,...,2.0,3.0,1.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0
9,2765,0,4.0,2.0,NaN,5.0,3.0,NaN,4.0,3.0,...,NaN,NaN,2.0,NaN,NaN,2.0,5.0,1.0,NaN,NaN


In [ ]:
# Calculate the mean for each movie, ignoring NaNs
mean_ratings = df.iloc[:, 2:].mean().sort_values(ascending=False)
mean_ratings.head(3)


318: Shawshank Redemption, The (1994)             3.600000
260: Star Wars: Episode IV - A New Hope (1977)    3.266667
541: Blade Runner (1982)                          3.222222
dtype: float64

In [13]:
num_ratings = df.iloc[:, 2:].count().sort_values(ascending=False)
num_ratings.head(3)

1: Toy Story (1995)                               17
593: Silence of the Lambs, The (1991)             16
260: Star Wars: Episode IV - A New Hope (1977)    15
dtype: int64

In [14]:
# Calculate the percentage of ratings 4 or higher for each movie
ratings = df.iloc[:, 2:]
percent_liking = (ratings.ge(4).sum() / ratings.count() * 100).sort_values(ascending=False) #.ge(4) is a boolean mask
percent_liking.head(3)


318: Shawshank Redemption, The (1994)             70.000000
260: Star Wars: Episode IV - A New Hope (1977)    53.333333
3578: Gladiator (2000)                            50.000000
dtype: float64

In [26]:
toy_story = [col for col in df.columns if col.startswith('1: Toy Story')][0] # [0] here means the first column that starts with '1: Toy Story'

raters = df[toy_story].notna()
other_cols = df.columns[2:]

pct = (df.loc[raters, other_cols].notna().sum() / raters.sum() * 100) #(x+y)/x

pct = pct.drop(toy_story)

# Sort by value descending, then by movie id ascending
pct = pct.sort_values(ascending=False)
pct = pct.loc[sorted(pct.index, key=lambda c: ( -pct[c], int(c.split(':')[0]) ))]

top2 = pct.head(4)

# Calculate correlations for the top two movies with Toy Story
correlations = {}
for col in top2.index:
    # Only consider users who rated both movies
    mask = df[toy_story].notna() & df[col].notna()
    if mask.sum() > 1:
        corr = df.loc[mask, [toy_story, col]].corr().iloc[0,1]
    else:
        corr = float('nan')
    correlations[col] = corr

# Display the top 3 as before
display = pct.head(4)
print(display)

# Add the correlations for the top two
print("\nCorrelations with Toy Story for the top two:")
for col in top2.index:
    print(f"{col}: {correlations[col]:.3f}")


260: Star Wars: Episode IV - A New Hope (1977)             82.352941
593: Silence of the Lambs, The (1991)                      76.470588
780: Independence Day (ID4) (1996)                         76.470588
1210: Star Wars: Episode VI - Return of the Jedi (1983)    76.470588
dtype: float64

Correlations with Toy Story for the top two:
260: Star Wars: Episode IV - A New Hope (1977): -0.119
593: Silence of the Lambs, The (1991): -0.568
780: Independence Day (ID4) (1996): -0.070
1210: Star Wars: Episode VI - Return of the Jedi (1983): -0.315


# this shows that not all people who watched and rated same movies liked it equaly and there is actually a negative correlation for the Silence of the Lambs - dislike

In [22]:
toy_story = [c for c in df.columns if c.startswith('1: Toy Story')][0]
other_cols = [c for c in df.columns[2:] if c != toy_story]
corr = {c: df.loc[df[toy_story].notna() & df[c].notna(), [toy_story, c]].corr().iloc[0,1]
        for c in other_cols if df[toy_story].notna().sum() > 1}
corr = pd.Series(corr)
corr = corr[corr > 0].sort_values(ascending=False)
print(corr.head(3))
print("\nCorrelations with Toy Story for the top two:")
for c in corr.head(2).index:
    print(f"{c}: {corr[c]:.3f}")



318: Shawshank Redemption, The (1994)    0.888523
34: Babe (1995)                          0.811107
296: Pulp Fiction (1994)                 0.709842
dtype: float64

Correlations with Toy Story for the top two:
318: Shawshank Redemption, The (1994): 0.889
34: Babe (1995): 0.811


## overlap shows what Toy Story fans also watched (popularity), while correlation shows what they liked in the same way (true taste similarity).

In [28]:
# Which movie do female raters rate the highest above male raters?
# And what are the average ratings for females and males for that movie?

# Identify gender column and movie columns
gender_col = [c for c in df.columns if 'Gender' in c][0]
movie_cols = df.columns[2:]  # assuming first two columns are User and Gender

# Compute mean rating for each movie by gender
mean_by_gender = df.groupby(gender_col)[movie_cols].mean().T

# Compute difference (female mean - male mean) for each movie
mean_diff = mean_by_gender[1] - mean_by_gender[0]  # 1 = F, 0 = M

# Find the movie with the largest difference (female - male)
movie_max_diff = mean_diff.idxmax()
diff_value = mean_diff.loc[movie_max_diff]
female_mean = mean_by_gender.loc[movie_max_diff, 1]
male_mean = mean_by_gender.loc[movie_max_diff, 0]

print(f"Movie where female raters rated highest above male raters:\n{movie_max_diff}")
print(f"Difference (female - male): {diff_value:.3f}")
print(f"Average rating by females: {female_mean:.3f}")
print(f"Average rating by males: {male_mean:.3f}")
# Find movies with greatest positive and negative differences
most_female_above = mean_diff.idxmax()
most_male_above = mean_diff.idxmin()

print("Movie where women rated most above men:")
print(f"{most_female_above}: {mean_diff[most_female_above]:.3f}")

print("\nMovie where men rated most above women:")
print(f"{most_male_above}: {mean_diff[most_male_above]:.3f}")

# Compute overall mean rating for males and females (across all ratings)
male_ratings = df.loc[df[gender_col] == 0, movie_cols].values.flatten()
female_ratings = df.loc[df[gender_col] == 1, movie_cols].values.flatten()

# Remove NaNs
male_ratings = male_ratings[~pd.isna(male_ratings)]
female_ratings = female_ratings[~pd.isna(female_ratings)]

overall_male_mean = male_ratings.mean()
overall_female_mean = female_ratings.mean()
overall_diff = overall_female_mean - overall_male_mean

print(f"\nOverall mean rating (female): {overall_female_mean:.3f}")
print(f"Overall mean rating (male): {overall_male_mean:.3f}")
print(f"Difference (female - male): {overall_diff:.3f}")


Movie where female raters rated highest above male raters:
2396: Shakespeare in Love (1998)
Difference (female - male): 2.107
Average rating by females: 4.250
Average rating by males: 2.143
Movie where women rated most above men:
2396: Shakespeare in Love (1998): 2.107

Movie where men rated most above women:
1198: Raiders of the Lost Ark (1981): -1.667

Overall mean rating (female): 2.947
Overall mean rating (male): 2.906
Difference (female - male): 0.042


On average, male and female ratings look almost identical.
But at the individual movie level, there can be big differences in preference.
Some movies are “gender-neutral” in appeal.
Others (like Shakespeare in Love or Raiders of the Lost Ark) show clear gender splits in taste.

Här är din kod uttryckt som matematiska formler.

**Notation**

* $M$: mängden filmer, index $i\in M$ (filmens id).
* $U_m$, $U_f$: mängden manliga respektive kvinnliga användare, index $u$.
* $r_{u i}$: betyg som användare $u$ gav film $i$; saknas $\Rightarrow$ odefinierat.
* $\mathbf{1}[\cdot]$: indikatorfunktion (1 om sant, annars 0).
* $a_{u i}=\mathbf{1}[r_{u i}\ \text{är definierat}]$ markerar att ett betyg finns.

**Andel 4+ per film och kön**

$$
p_i^{(m)}=\frac{\sum_{u\in U_m}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_m} a_{u i}},
\qquad
p_i^{(f)}=\frac{\sum_{u\in U_f}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_f} a_{u i}}.
$$

**Skillnad (kvinna − man) per film**

$$
\Delta_i \;=\; p_i^{(f)}-p_i^{(m)}.
$$

**Filmer med största positiva/negativa skillnaden**

$$
i^{+}=\arg\max_{i\in M}\Delta_i,\qquad
i^{-}=\arg\min_{i\in M}\Delta_i.
$$

*(Vid lika värden kan tie-break göras med minsta film-id: välj $ \min\{i:\Delta_i=\max_j \Delta_j\}$, analogt för $\min$).*

**Övergripande andel 4+ per kön (över alla filmer)**

$$
P^{(m)}=\frac{\sum_{u\in U_m}\sum_{i\in M}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_m}\sum_{i\in M} a_{u i}},
\qquad
P^{(f)}=\frac{\sum_{u\in U_f}\sum_{i\in M}\mathbf{1}[\,r_{u i}\ge 4\,]}{\sum_{u\in U_f}\sum_{i\in M} a_{u i}}.
$$

**Övergripande skillnad**

$$
\Delta_{\text{overall}} \;=\; P^{(f)}-P^{(m)}.
$$

Detta motsvarar exakt din beräkning: andelar av 4+ per film/kön, differenserna, extremfilmerna samt total skillnad mellan könen.


In [32]:
import re
import pandas as pd

# --- your existing inputs assumed: df, gender_col, movie_cols ---

# For each gender, boolean DataFrame: True if rating >= 4 (ignoring NaN)
male_4plus = (df.loc[df[gender_col] == 0, movie_cols] >= 4)
female_4plus = (df.loc[df[gender_col] == 1, movie_cols] >= 4)

# % of ratings 4+ for each movie (True count / non-NaN count)
male_4plus_pct = male_4plus.sum() / (~df.loc[df[gender_col] == 0, movie_cols].isna()).sum()
female_4plus_pct = female_4plus.sum() / (~df.loc[df[gender_col] == 1, movie_cols].isna()).sum()

# Difference (female % - male %) for each movie
pct_diff = (female_4plus_pct - male_4plus_pct).dropna()

# --- tie-break by lowest movie id ---
def movie_id(col):
    return int(str(col).split(':', 1)[0])

diff_df = pct_diff.to_frame('diff')
diff_df['id'] = diff_df.index.map(movie_id)

# Women > men (max diff; tie -> lowest id)
most_female_above_4plus = diff_df.sort_values(['diff', 'id'], ascending=[False, True]).index[0]
# Men > women (most negative diff; tie -> lowest id)
most_male_above_4plus = diff_df.sort_values(['diff', 'id'], ascending=[True, True]).index[0]

print("Movie where women are most more likely to rate 4+ than men:")
print(f"{most_female_above_4plus}: {pct_diff[most_female_above_4plus]:.3f}")

print("\nMovie where men are most more likely to rate 4+ than women:")
print(f"{most_male_above_4plus}: {pct_diff[most_male_above_4plus]:.3f}")

# --- overall % of 4+ across all movies (build from actual ratings vectors) ---
male_ratings = df.loc[df[gender_col] == 0, movie_cols].stack().dropna()
female_ratings = df.loc[df[gender_col] == 1, movie_cols].stack().dropna()

overall_male_4plus = (male_ratings >= 4).mean()
overall_female_4plus = (female_ratings >= 4).mean()
overall_4plus_diff = overall_female_4plus - overall_male_4plus

print(f"\nOverall % of 4+ ratings (female): {overall_female_4plus:.3f}")
print(f"Overall % of 4+ ratings (male):   {overall_male_4plus:.3f}")
print(f"Difference (female - male):       {overall_4plus_diff:.3f}")


Movie where women are most more likely to rate 4+ than men:
2396: Shakespeare in Love (1998): 0.750

Movie where men are most more likely to rate 4+ than women:
1198: Raiders of the Lost Ark (1981): -0.500

Overall % of 4+ ratings (female): 0.421
Overall % of 4+ ratings (male):   0.339
Difference (female - male):       0.082


In [2]:
import pandas as pd

# Read assignment2.csv from the data/external directory
assignment2 = pd.read_csv('../data/external/assignment2.csv')
assignment2.tail()


,doc,baseball,economics,politics,Europe,Asia,soccer,war,security,shopping,family,num-attr,User1,User2
15,doc16,1,0,0,0,0,1,0,0,1,0,3,1.0,NaN
16,doc17,0,1,1,1,0,0,0,1,0,0,4,NaN,1.0
17,doc18,0,0,0,1,0,0,0,0,1,0,2,NaN,NaN
18,doc19,0,1,1,0,1,0,1,0,0,1,5,-1.0,NaN
19,doc20,0,0,1,1,0,0,1,0,1,0,4,NaN,NaN


In [3]:
# Step 2: Define columns and basic cleaning

# List your attribute columns exactly as in the CSV (order matters and must match the CSV)
attr_cols = [
    'baseball','economics','politics','Europe','Asia',
    'soccer','war','security','shopping','family'
]

# Ratings columns
user_cols = ['User1','User2']

# Ensure ratings are numeric with unrated -> 0
assignment2[user_cols] = assignment2[user_cols].fillna(0).astype(int)

# (Optional) sanity check: num-attr equals row sum of attributes
assignment2['num-attr_check'] = assignment2[attr_cols].sum(axis=1)


In [4]:
import numpy as np

# Step 2: Define columns and basic cleaning

# List your attribute columns exactly as in the CSV (order matters and must match the CSV)
attr_cols = [
    'baseball','economics','politics','Europe','Asia',
    'soccer','war','security','shopping','family'
]

# Ratings columns
user_cols = ['User1','User2']

# Ensure ratings are numeric with unrated -> 0
assignment2[user_cols] = assignment2[user_cols].fillna(0).astype(int)

# (Optional) sanity check: num-attr equals row sum of attributes
assignment2['num-attr_check'] = assignment2[attr_cols].sum(axis=1)


In [5]:
# Step 3: Build user profiles (per-user, per-attribute)
# profile[u, a] = sum over docs of rating(u, d) * has_attribute(d, a)

X = assignment2[attr_cols].to_numpy(dtype=int)          # docs x attributes
profiles = {}

for u in user_cols:
    r = assignment2[u].to_numpy(dtype=int)              # docs vector
    p = X.T @ r                                         # attributes vector
    profiles[u] = p

# Turn into a tidy DataFrame: rows = attributes, cols = users
user_profiles = pd.DataFrame(profiles, index=attr_cols)
user_profiles.index.name = 'attribute'
user_profiles


,User1,User2
attribute,,
baseball,3,-2
economics,-2,2
politics,-1,2
Europe,0,3
Asia,0,-1
soccer,2,-2
war,-1,0
security,-1,3
shopping,1,0


In [6]:
# Step 4: Predict scores for each doc (simple dot product)
# score(u, d) = user_profile(u) dot attributes(d)

pred_cols = {}
for u in user_cols:
    p = user_profiles[u].to_numpy()                     # attributes vector
    scores = X @ p                                      # docs vector
    pred_col = f'pred_{u}'
    assignment2[pred_col] = scores
    pred_cols[u] = pred_col

# Show a compact view: doc, ratings, predictions
assignment2[['doc','User1','User2', pred_cols['User1'], pred_cols['User2']]].head()

,doc,User1,User2,pred_User1,pred_User2
0,doc1,1,-1,4,-4
1,doc2,-1,1,-4,10
2,doc3,0,0,2,0
3,doc4,0,1,-3,8
4,doc5,0,0,-1,1


In [7]:
# Step 5: Answer the Quiz

# Q1. Which document does the simple profile predict user 1 will like best?
best_idx_u1 = assignment2[pred_cols['User1']].idxmax()
best_doc_u1 = assignment2.loc[best_idx_u1, 'doc']

# Q2. What score does that prediction get?
best_score_u1 = int(assignment2.loc[best_idx_u1, pred_cols['User1']])

# Q3. How many documents does the model predict user 2 will dislike (score < 0)?
num_disliked_u2 = int((assignment2[pred_cols['User2']] < 0).sum())

print("Quiz Answers")
print("------------")
print(f"Q1: Document predicted best for User 1: {best_doc_u1}")
print(f"Q2: Score of that prediction: {best_score_u1}")
print(f"Q3: # of documents User 2 is predicted to dislike: {num_disliked_u2}")


Quiz Answers
------------
Q1: Document predicted best for User 1: doc16
Q2: Score of that prediction: 6
Q3: # of documents User 2 is predicted to dislike: 4


In [8]:
# Step 6 (Optional): Verify consistency with labeled ratings
# The simple profile should predict positive for all +1 ratings and negative for all -1 ratings.

def consistency_report(user_col, pred_col):
    liked = assignment2[assignment2[user_col] == 1]
    disliked = assignment2[assignment2[user_col] == -1]
    liked_correct = int((liked[pred_col] > 0).sum())
    disliked_correct = int((disliked[pred_col] < 0).sum())
    return {
        'user': user_col,
        'liked_docs': int(len(liked)),
        'liked_predicted_positive': liked_correct,
        'disliked_docs': int(len(disliked)),
        'disliked_predicted_negative': disliked_correct
    }

consistency = pd.DataFrame([
    consistency_report('User1', pred_cols['User1']),
    consistency_report('User2', pred_cols['User2'])
])
consistency


,user,liked_docs,liked_predicted_positive,disliked_docs,disliked_predicted_negative
0,User1,3,3,2,2
1,User2,3,3,2,2


Part 2

In [9]:
# --- 3) Row-normalize the attributes (unit length per doc) --------------------
# X: docs x attributes
X = assignment2[attr_cols].to_numpy(dtype=float)

# k: number of attributes per doc (already provided as num-attr in your CSV)
k = assignment2['num-attr'].to_numpy(dtype=float)

# Divide each row by sqrt(k)
Xn = X / np.sqrt(k)[:, None]

# --- 4) Build normalized profiles (sum of rating * normalized attributes) -----
p_user = {}
for u in user_cols:
    r = assignment2[u].to_numpy()
    p_user[u] = Xn.T @ r      # attributes vector per user

# --- 5) Predict scores (dot product of profile and normalized doc attributes) -
for u in user_cols:
    assignment2[f'pred_{u}_norm'] = Xn @ p_user[u]

# --- 6) Spot-checks from the instructions ------------------------------------
u1_doc1 = assignment2.loc[assignment2['doc']=='doc1', 'pred_User1_norm'].values[0]
u2_doc7 = assignment2.loc[assignment2['doc']=='doc7', 'pred_User2_norm'].values[0]
u2_doc19 = assignment2.loc[assignment2['doc']=='doc19','pred_User2_norm'].values[0]

print("Spot-checks (should be close to the given targets):")
print(f"User1 • doc1  ≈ 1.009  -> {u1_doc1:.4f}")
print(f"User2 • doc7  ≈ 0.7444 -> {u2_doc7:.4f}")
print(f"User2 • doc19 ≈ 0.4834 -> {u2_doc19:.4f}")

# --- 7) Quiz answers ----------------------------------------------------------
# For User 1, find the second-place document under the new model
u1_ranked = assignment2[['doc','pred_User1_norm']].sort_values('pred_User1_norm', ascending=False).reset_index(drop=True)
first_place_doc  = u1_ranked.loc[0, 'doc']
second_place_doc = u1_ranked.loc[1, 'doc']
second_place_val = u1_ranked.loc[1, 'pred_User1_norm']

print("\nQuiz answers (normalized model):")
print(f"- User1 second-place document: {second_place_doc}")
print(f"- Its prediction score: {second_place_val:.4f}")

# (Optional) peek at the top 10 for User1
print("\nTop 10 (User1, normalized):")
print(u1_ranked.head(10).to_string(index=False))

Spot-checks (should be close to the given targets):
User1 • doc1  ≈ 1.009  -> 1.0090
User2 • doc7  ≈ 0.7444 -> 0.7444
User2 • doc19 ≈ 0.4834 -> 0.4834

Quiz answers (normalized model):
- User1 second-place document: doc6
- Its prediction score: 1.3709

Top 10 (User1, normalized):
  doc  pred_User1_norm
doc16         1.924646
 doc6         1.370923
doc12         1.333114
 doc9         1.132724
 doc1         1.009019
 doc3         0.711105
doc18         0.554695
doc15         0.142229
doc11         0.044658
doc20        -0.081378


Part3: 

In [11]:
import pandas as pd
import numpy as np

# --- Load data ---------------------------------------------------------------
assignment2 = pd.read_csv('../data/external/assignment2.csv')

# Columns
attr_cols = [
    'baseball','economics','politics','Europe','Asia',
    'soccer','war','security','shopping','family'
]
user_cols = ['User1','User2']

# Ensure ratings are numeric, unrated -> 0
assignment2[user_cols] = assignment2[user_cols].fillna(0).astype(int)

# --- Part 2 recap: row-normalize attributes (unit length per doc) -----------
X = assignment2[attr_cols].to_numpy(dtype=float)        # docs x attributes (0/1)
k = assignment2['num-attr'].to_numpy(dtype=float)       # number of attrs per doc
Xn = X / np.sqrt(k)[:, None]                            # normalized rows

# Build normalized profiles from Part 2
profiles_norm = {}
for u in user_cols:
    r = assignment2[u].to_numpy()                       # docs vector
    profiles_norm[u] = Xn.T @ r                         # attributes vector

# Part 2 predictions (for comparison later)
assignment2['pred_User1_norm'] = Xn @ profiles_norm['User1']
assignment2['pred_User2_norm'] = Xn @ profiles_norm['User2']

# --- Part 3: IDF = 1/DF ------------------------------------------------------
# DF[a] = # of documents where attribute a == 1  (use the original 0/1 X)
DF = X.sum(axis=0)                                      # length = #attributes
IDF = 1.0 / DF                                          # 1/DF per attribute

idf_series = pd.Series(IDF, index=attr_cols, name='IDF')

# --- IDF-weighted predictions: doc · profile · IDF --------------------------
# For each user u: score_idf(d) = sum_a  Xn[d,a] * profiles_norm[u][a] * IDF[a]
# Implement as:   scores_idf = Xn @ (profiles_norm[u] * IDF)
pred_idf = {}
for u in user_cols:
    w = profiles_norm[u] * IDF                          # attribute weights with IDF
    pred = Xn @ w                                       # one score per doc
    assignment2[f'pred_{u}_idf'] = pred
    pred_idf[u] = pred

# --- Spot-check expected value from prompt -----------------------------------
u1_doc1_idf = assignment2.loc[assignment2['doc']=='doc1', 'pred_User1_idf'].values[0]

print("Spot-check (should be ≈ 0.2476 for User1/doc1):")
print(f"User1 • doc1 (IDF-weighted): {u1_doc1_idf:.4f}")

# --- Quiz comparisons ---------------------------------------------------------
# 1) Compare doc1 and doc9 for User1 under IDF-weighted model
u1_doc9_idf = assignment2.loc[assignment2['doc']=='doc9', 'pred_User1_idf'].values[0]
u1_doc9_norm = assignment2.loc[assignment2['doc']=='doc9', 'pred_User1_norm'].values[0]

print("\nUser1 comparisons:")
print(f"- doc1 (IDF): {u1_doc1_idf:.4f}")
print(f"- doc9 (IDF): {u1_doc9_idf:.4f}")
print(f"  (Previously, doc9 under Part-2 normalization: {u1_doc9_norm:.4f})")

# 2) Explain why User2/doc6 flips from moderately positive to slightly negative
u2_doc6_norm = assignment2.loc[assignment2['doc']=='doc6', 'pred_User2_norm'].values[0]
u2_doc6_idf  = assignment2.loc[assignment2['doc']=='doc6', 'pred_User2_idf'].values[0]

print("\nUser2/doc6 shift:")
print(f"- Part 2 (normalized) prediction: {u2_doc6_norm:.4f}")
print(f"- Part 3 (IDF-weighted) prediction: {u2_doc6_idf:.4f}")

# Give a short reason based on which attributes doc6 has and their IDF weights
doc6_attrs = assignment2.loc[assignment2['doc']=='doc6', attr_cols].iloc[0]
present_attrs = [a for a in attr_cols if doc6_attrs[a] == 1]
print(f"\nDoc6 attributes: {present_attrs}")
print("IDF weights for Doc6 attributes:")
for a in present_attrs:
    print(f"  - {a}: IDF = {idf_series[a]:.4f}")

# Also show User2's profile weights on those attributes (pre-IDF, from Part 2)
p2 = pd.Series(profiles_norm['User2'], index=attr_cols, name='User2_profile_norm')
print("\nUser2 profile (normalized) on Doc6 attributes (before IDF):")
for a in present_attrs:
    print(f"  - {a}: {p2[a]:.4f}")

# And the per-attribute contributions under IDF weighting (for doc6)
row6 = Xn[assignment2.index[assignment2['doc']=='doc6'][0], :]  # normalized row for doc6
contrib = pd.Series(row6 * p2.values * idf_series.values, index=attr_cols)
print("\nPer-attribute contributions to User2/doc6 (IDF-weighted):")
for a in present_attrs:
    print(f"  - {a}: {contrib[a]:.4f}")

# --- Final succinct answers ---------------------------------------------------
print("\nQuiz answers (Part 3, IDF-weighted):")
print(f"- User1: prediction for doc9 (IDF-weighted) = {u1_doc9_idf:.4f}")
print(f"- Why User2/doc6 changed sign: IDF heavily downweights common attributes "
      f"(e.g., 'Europe' occurs in many docs), so User2's positive signal on common topics "
      f"is reduced, while a negative/weak signal on a rarer topic (e.g., 'baseball' with higher IDF) "
      f"can dominate, nudging the total slightly below zero.")


Spot-check (should be ≈ 0.2476 for User1/doc1):
User1 • doc1 (IDF-weighted): 0.2476

User1 comparisons:
- doc1 (IDF): 0.2476
- doc9 (IDF): 0.1791
  (Previously, doc9 under Part-2 normalization: 1.1327)

User2/doc6 shift:
- Part 2 (normalized) prediction: 0.3362
- Part 3 (IDF-weighted) prediction: -0.0847

Doc6 attributes: ['baseball', 'Europe']
IDF weights for Doc6 attributes:
  - baseball: IDF = 0.2500
  - Europe: IDF = 0.0909

User2 profile (normalized) on Doc6 attributes (before IDF):
  - baseball: -1.0246
  - Europe: 1.5000

Per-attribute contributions to User2/doc6 (IDF-weighted):
  - baseball: -0.1811
  - Europe: 0.0964

Quiz answers (Part 3, IDF-weighted):
- User1: prediction for doc9 (IDF-weighted) = 0.1791
- Why User2/doc6 changed sign: IDF heavily downweights common attributes (e.g., 'Europe' occurs in many docs), so User2's positive signal on common topics is reduced, while a negative/weak signal on a rarer topic (e.g., 'baseball' with higher IDF) can dominate, nudging the t